# Large Matrix Algebra Broadcasting

Back to [**Fan**](http://fanwangecon.github.io)'s [**Matlab Examples Table of Content**](https://fanwangecon.github.io/M4Econ/)

Addition, subtraction, these basic algebra tasks are extremely fast. 

But creating new matrixes are slow.

Matrix [broadcasting](https://www.mathworks.com/help/matlab/matlab_prog/compatible-array-sizes-for-basic-operations.html), which matlab calls [implicit expansion](https://blogs.mathworks.com/loren/2016/10/24/matlab-arithmetic-expands-in-r2016b/) was introduced in matlab r2016b. 

Let's broacast and do arithmetics together. 

## Broadcast Combine States and Choices

We will use a state space vector for column values, and choice vector as row values. This should be done this way for fast maximization because matlab is a [column-major](https://en.wikipedia.org/wiki/Row-_and_column-major_order) language. 

Broacasting is a very basic idea, we can add states and choices together without replicating matrixes initially. This improves memory usage potentially significantly when we deal with large matrixes

In [1]:
clear all
% Here are our state and choice vectors
it_cash_on_hand_n = 10;
ar_cash_on_hand = linspace(1,10,it_cash_on_hand_n)
it_b_choice_n = 5;
ar_b_choice = linspace(1,10,it_b_choice_n)
disp(ar_cash_on_hand);
disp(ar_b_choice);


ar_cash_on_hand =

     1     2     3     4     5     6     7     8     9    10


ar_b_choice =

    1.0000    3.2500    5.5000    7.7500   10.0000

     1     2     3     4     5     6     7     8     9    10

    1.0000    3.2500    5.5000    7.7500   10.0000




In [2]:
% broacast together to generate all combinations of states and choices
ar_cash_on_hand - ar_b_choice'


ans =

         0    1.0000    2.0000    3.0000    4.0000    5.0000    6.0000    7.0000    8.0000    9.0000
   -2.2500   -1.2500   -0.2500    0.7500    1.7500    2.7500    3.7500    4.7500    5.7500    6.7500
   -4.5000   -3.5000   -2.5000   -1.5000   -0.5000    0.5000    1.5000    2.5000    3.5000    4.5000
   -6.7500   -5.7500   -4.7500   -3.7500   -2.7500   -1.7500   -0.7500    0.2500    1.2500    2.2500
   -9.0000   -8.0000   -7.0000   -6.0000   -5.0000   -4.0000   -3.0000   -2.0000   -1.0000         0




## Initialize

In [3]:
% Let's have larger matrixes
it_b_choice_n = 300;
it_cash_on_hand_n = round(((it_b_choice_n-1)*it_b_choice_n)/2 + it_b_choice_n);
ar_cash_on_hand = linspace(1,10,it_cash_on_hand_n);
ar_b_choice = linspace(1,10,it_b_choice_n);
ar_k_choice = 10 - ar_b_choice;

In [4]:
% ar_fl_exe_times = {};
% ar_fl_exe_desc = {};
it_coll = 0;
z = 15;
iter = 50;

## Broadcasting Speed
Now, let's use larger matrixes, and run our broadcast method many times. 

In [5]:
% Interpolation Evaluator
f_broadcast_algebra = @() ar_cash_on_hand - ar_b_choice'/(1+0.02) - ar_k_choice';
size(f_broadcast_algebra())


ans =

         300       45150




In [6]:
fl_exe_time = timeit(f_broadcast_algebra);
it_coll = it_coll + 1;
ar_fl_exe_times(it_coll) = fl_exe_time;
ar_fl_exe_desc{it_coll} = 'Broadcast from Vectors';

## Broadcasting Speed With Initialized Matrix.
Same thing as before, but let's see if speed improves when we initialize the full matrix and add to that. 

In [7]:
mt_zero_init = zeros([it_b_choice_n, it_cash_on_hand_n]);

In [8]:
% Interpolation Evaluator
f_broadcast_algebra_initialized = @() mt_zero_init + ar_cash_on_hand - ar_b_choice'/(1+0.02) - ar_k_choice';

In [9]:
fl_exe_time = timeit(f_broadcast_algebra_initialized);
it_coll = it_coll + 1;
ar_fl_exe_times(it_coll) = fl_exe_time;
ar_fl_exe_desc{it_coll} = 'Broadcast from Zero Matrix Plus Vectors';

## Broadcasting Speed With Initialized Matrix Equal to Existing

We will now use two functions. The difference is wehtehr the output matrix is initialized first. 

In [10]:
% function [mt_c] =  ff_broadcast_mat(ar_cash_on_hand, ar_b_choice, ar_k_choice)
% % Assume ar_b_choice and ar_k_choice are N by 1
% % Assume ar_cash_on_hand and ar_k_choice are 1 by M
%     mt_c = ar_cash_on_hand - ar_b_choice/(1+0.02) - ar_k_choice;
% end

% function [mt_c] =  ff_broadcast_zeros(ar_cash_on_hand, ar_b_choice, ar_k_choice)
% % Assume ar_b_choice and ar_k_choice are N by 1
% % Assume ar_cash_on_hand and ar_k_choice are 1 by M
%     mt_c = zeros([length(ar_b_choice), length(ar_cash_on_hand)]);
%     mt_c = mt_c +  ar_cash_on_hand - ar_b_choice/(1+0.02) - ar_k_choice;
% end

In [11]:
% Evaluators
f_broadcast_alg_newmat_func = @() ff_broadcast_mat(ar_cash_on_hand, ar_b_choice', ar_k_choice');
f_broadcast_alg_initialize_func = @() ff_broadcast_zeros(ar_cash_on_hand, ar_b_choice', ar_k_choice');

In [12]:
fl_exe_time = timeit(f_broadcast_alg_newmat_func);
it_coll = it_coll + 1;
ar_fl_exe_times(it_coll) = fl_exe_time;
ar_fl_exe_desc{it_coll} = 'Broadcast Function Without Initialization';

In [13]:
fl_exe_time = timeit(f_broadcast_alg_initialize_func);
it_coll = it_coll + 1;
ar_fl_exe_times(it_coll) = fl_exe_time;
ar_fl_exe_desc{it_coll} = 'Broadcast Function with empty Initialization';

## Timing Results

In [14]:
tb_exe_times = array2table([ar_fl_exe_times', ar_fl_exe_times'*z*iter]);
tb_exe_times.Properties.RowNames = ar_fl_exe_desc;
tb_exe_times.Properties.VariableNames = {'speedmat', 'speedfull'};
disp(tb_exe_times);

                                                    speedmat    speedfull
                                                    ________    _________

    Broadcast from Vectors                          0.070854      53.14  
    Broadcast from Zero Matrix Plus Vectors         0.073897     55.423  
    Broadcast Function Without Initialization        0.03464      25.98  
    Broadcast Function with empty Initialization    0.037027      27.77  


